# Cleaning and Preprocessing Sales Data with Pandas

This notebook demonstrates the automated cleaning and preprocessing of sales data using Python and the Pandas library. The goal is to prepare raw sales data for further analysis or reporting by addressing inconsistencies, errors, and missing values.

## Key Steps Performed in the Notebook:
1. **Data Loading**: Reading the raw sales data
2. **Exploratory Data Analysis (EDA)**:
   - Displaying the dataset's structure (e.g., columns, data types, and sample records).
   - Identifying and summarizing missing or erroneous data.
3. **Data Cleaning**:
   - Removing or imputing missing values.
   - Baisc data maksing customer code
   - Handling duplicate entries.
   - Correcting inconsistent or invalid data (e.g., standardizing text formats or fixing outliers).
4. **Data Transformation**:
   - Renaming columns for better clarity and consistency.
   - Converting data types (e.g., dates, numerical values).
   - Calculating new fields, such as totals, discounts, or derived metrics.
5. **Validation**:
   - Verifying that the cleaned dataset meets quality standards.
   - Ensuring the output is ready for downstream applications or analysis.



In [1]:
import pandas as pd  # Importing the Pandas library for data manipulation and analysis



In [2]:
# Import customer data from a CSV file into a DataFrame
customers = pd.read_csv('customers.csv')

# Import date data from a CSV file into a DataFrame
date = pd.read_csv('date.csv')

# Import market data from a CSV file into a DataFrame
markets = pd.read_csv('markets.csv')

# Import product data from a CSV file into a DataFrame
products = pd.read_csv('products.csv')

# Import transaction data from a CSV file into a DataFrame
transactions = pd.read_csv('transactions.csv')


In [20]:
# Print the shape (number of rows and columns) of the customers table
print(f"customers_table shape: {customers.shape}")

# Print the shape (number of rows and columns) of the date table
print(f"date_table shape: {date.shape}")

# Print the shape (number of rows and columns) of the markets table
print(f"markets_table shape: {markets.shape}")

# Print the shape (number of rows and columns) of the products table
print(f"products_table shape: {products.shape}")

# Print the shape (number of rows and columns) of the transactions table
print(f"transactions_table shape: {transactions.shape}")



customers_table shape: (38, 3)
date_table shape: (1126, 5)
markets_table shape: (17, 3)
products_table shape: (279, 2)
transactions_table shape: (148395, 10)


In [3]:


# Calculate the number of null (missing) values in each column of the customers DataFrame
null = customers.isnull().sum()
print(f"Number of Null values in customers:\n{null}")

# Calculate the number of duplicate rows in the customers DataFrame
duplicates = customers.duplicated().sum()
print(f"Number of duplicate rows in customers: {duplicates}")



# Create a dictionary to map original customer codes to their hashed values from the customers DataFrame
customer_code_map = dict(zip(customers['customer_code'], customers['customer_code'].apply(lambda x: hash(x))))


# Check and display the data types of each column in the customers DataFrame
print("Data types of the columns in customers DataFrame:")
print(customers.dtypes)

# Display the first few rows of the customers DataFrame for a quick overview
customers.head()




Number of Null values in customers:
customer_code    0
custmer_name     0
customer_type    0
dtype: int64
Number of duplicate rows in customers: 0
Data types of the columns in customers DataFrame:
customer_code    object
custmer_name     object
customer_type    object
dtype: object


,customer_code,custmer_name,customer_type
0,Cus001,Surge Stores,Brick & Mortar
1,Cus002,Nomad Stores,Brick & Mortar
2,Cus003,Excel Stores,Brick & Mortar
3,Cus004,Surface Stores,Brick & Mortar
4,Cus005,Premium Stores,Brick & Mortar


In [22]:


# Calculate the number of null (missing) values in each column of the dates DataFrame
null = date.isnull().sum()
print(f"Number of Null values in dates:\n {null}")

# Drop unnecessary columns ('cy_date' and 'date_yy_mmm') from the dates DataFrame
date.drop(columns=['cy_date', 'date_yy_mmm'], inplace=True)

# Change the datatype of the 'date' column to datetime for proper date manipulation
date['date'] = pd.to_datetime(date['date'])

# Display the data types of each column in the dates DataFrame to confirm the changes
print("Data types of the columns in dates DataFrame:")
print(date.dtypes)



Number of Null values in dates:
 date           0
cy_date        0
year           0
month_name     0
date_yy_mmm    0
dtype: int64
Data types of the columns in dates DataFrame:
date          datetime64[ns]
year                   int64
month_name            object
dtype: object


In [23]:
# Calculate the number of null (missing) values in each column of the markets DataFrame
null = markets.isnull().sum()
print(f"Number of Null values in markets:\n {null}")

# Display the data types of each column in the markets DataFrame
print("Data types of the columns in markets DataFrame:")
print(markets.dtypes)

# After checking in SQL, we found two unrelated market_name entries, so we drop rows with missing values
markets = markets.dropna()


Number of Null values in markets:
 markets_code    0
markets_name    0
zone            2
dtype: int64
Data types of the columns in markets DataFrame:
markets_code    object
markets_name    object
zone            object
dtype: object


In [24]:
# Calculate the number of null (missing) values in each column of the products DataFrame
null = products.isnull().sum()
print(f"Number of Null values in products:\n {null}")

# Calculate the number of duplicate rows in the products DataFrame
duplicates = products.duplicated().sum()
print(f"number of duplicates in products:{duplicates}")

# Display the data types of each column in the products DataFrame
print("Data types of the columns in products DataFrame:")
print(products.dtypes)

# Remove any extra carriage return characters ('\r') from the 'product_type' column
products['product_type'] = products['product_type'].str.strip()

# Display the first few rows of the products DataFrame for a quick overview
products.head()


Number of Null values in products:
 product_code    0
product_type    0
dtype: int64
number of duplicates in products:0
Data types of the columns in products DataFrame:
product_code    object
product_type    object
dtype: object


,product_code,product_type
0,Prod001,Own Brand
1,Prod002,Own Brand
2,Prod003,Own Brand
3,Prod004,Own Brand
4,Prod005,Own Brand


In [25]:
# Calculate the number of null (missing) values in each column of the transactions DataFrame
null = transactions.isnull().sum()
print(f"null values in transactions:{null}")

# Convert the 'order_date' column to datetime datatype for proper date manipulation
transactions['order_date'] = pd.to_datetime(transactions['order_date'])
print(transactions.dtypes)

# Ensure 'sales_amount' is greater than 0, replacing any non-positive values with 0
transactions['sales_amount'] = transactions['sales_amount'].apply(lambda sales: sales if sales > 0 else 0)

# Filter out rows where 'sales_amount' is 0 or negative
transactions = transactions[transactions['sales_amount'] > 0]

# Get the unique values in the 'currency' column
unique_currencies = transactions['currency'].unique()
print(f"Unique values in currency: {unique_currencies}")

# Remove any extra spaces from the 'currency' column
transactions['currency'].str.strip()

# Create a new column 'normal_sales_amount' that converts sales amounts in USD to the equivalent value in another currency (assumed exchange rate of 75)
transactions['normal_sales_amount'] = transactions.apply(lambda row: row['sales_amount'] * 75 if row['currency'] == 'USD' else row['sales_amount'], axis=1)

# Mask the 'customer_code' column for privacy by replacing it with a hashed value


# Apply the same masking to the 'customer_code' column in the transactions DataFrame
transactions['customer_code'] = transactions['customer_code'].map(customer_code_map)

# Display the first few rows of the transactions DataFrame for a quick overview
transactions.head()

null values in transactions:product_code                0
customer_code               0
market_code                 0
order_date                  0
sales_qty                   0
sales_amount                0
currency                    0
profit_margin_percentage    0
profit_margin               0
cost_price                  0
dtype: int64
product_code                        object
customer_code                       object
market_code                         object
order_date                  datetime64[ns]
sales_qty                            int64
sales_amount                         int64
currency                            object
profit_margin_percentage           float64
profit_margin                      float64
cost_price                         float64
dtype: object
Unique values in currency: ['INR' 'USD']


,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price,normal_sales_amount
0,Prod279,8597356859053196417,Mark011,2017-10-11,1,102,INR,0.39,39.78,62.22,102
1,Prod279,8597356859053196417,Mark011,2017-10-18,1,102,INR,-0.12,-12.24,114.24,102
2,Prod279,8597356859053196417,Mark011,2017-10-19,1,102,INR,0.29,29.58,72.42,102
3,Prod279,8597356859053196417,Mark011,2017-11-08,1,102,INR,0.36,36.72,65.28,102
4,Prod279,8597356859053196417,Mark011,2018-03-09,1,102,INR,-0.35,-35.70,137.70,102


**exporting cleaned data**

In [26]:
# Define the file path where the cleaned data will be saved
file_path = "C:\\Users\\Agasthya\\Documents\\data analysis\\sales_data analysis\\cleaned_data"

# Save the cleaned 'customers' DataFrame to a CSV file without including the index
customers.to_csv(f'{file_path}\\customers.csv', index=False)

# Save the cleaned 'date' DataFrame to a CSV file without including the index
date.to_csv(f'{file_path}\\date.csv', index=False)

# Save the cleaned 'markets' DataFrame to a CSV file without including the index
markets.to_csv(f'{file_path}\\markets.csv', index=False)

# Save the cleaned 'products' DataFrame to a CSV file without including the index
products.to_csv(f'{file_path}\\products.csv', index=False)

# Save the cleaned 'transactions' DataFrame to a CSV file without including the index
transactions.to_csv(f'{file_path}\\transactions.csv', index=False)


## Output:
The cleaned and transformed dataset is saved in a specified format (e.g., CSV or Excel) for seamless integration into analytics workflows.

This notebook serves as a reusable template for cleaning sales data, allowing for customization based on specific dataset requirements.